In [32]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [34]:
url = 'https://www.pro-football-reference.com'
year = 2020
maxp = 300
    
# grab fantasy players
r = requests.get(url + '/years/' + str(year) + '/fantasy.htm')
soup = BeautifulSoup(r.content, 'html.parser')
parsed_table = soup.find_all('table')[0]  

df = []

# first 2 rows are col headers
for i,row in enumerate(parsed_table.find_all('tr')[2:]):
    if i % 10 == 0: print(i, end=' ')
    if i >= maxp: 
        print('\nComplete.')
        break
        
    try:
        dat = row.find('td', attrs={'data-stat': 'player'})
        name = dat.a.get_text()
        stub = dat.a.get('href')
        stub = stub[:-4] + '/fantasy/' + str(year)
        pos = row.find('td', attrs={'data-stat': 'fantasy_pos'}).get_text()

        # grab this players stats
        tdf = pd.read_html(url + stub)[0]    

        # get rid of MultiIndex, just keep last row
        tdf.columns = tdf.columns.get_level_values(-1)

        # fix the away/home column
        tdf = tdf.rename(columns={'Unnamed: 4_level_2': 'Away'})
        tdf['Away'] = [1 if r=='@' else 0 for r in tdf['Away']]

        # drop all intermediate stats
        tdf = tdf.iloc[:,[1,2,3,4,5,-3]]
        
        # drop "Total" row
        tdf = tdf.query('Date != "Total"')
        
        # add other info
        tdf['Name'] = name
        tdf['Position'] = pos
        tdf['Season'] = year

        df.append(tdf)
    except:
        pass
    
df = pd.concat(df)
df.head()

0 10 20 30 40 50 60 70 80 90 100 110 120 130 140 150 160 170 180 190 200 210 220 230 240 250 260 270 280 290 300 
Complete.


,G#,Date,Tm,Away,Opp,FantPt,Name,Position,Season
0,1.0,2020-09-14,TEN,1,DEN,13.1,Derrick Henry,RB,2020
1,2.0,2020-09-20,TEN,0,JAX,8.4,Derrick Henry,RB,2020
2,3.0,2020-09-27,TEN,1,MIN,25.0,Derrick Henry,RB,2020
3,4.0,2020-10-13,TEN,0,BUF,18.3,Derrick Henry,RB,2020
4,5.0,2020-10-18,TEN,0,HOU,38.4,Derrick Henry,RB,2020


In [36]:
df.to_csv('fantasy2020.csv')